# <center><font color='blue'>SENTIMENT ANALYSIS: COVID</center></font>

## Tabla de contenido
- [1 - Objetivos](#1)
- [2 - Librerías necesarias](#2)
- [3 - Carga y visualización de datos](#3)
- [4 - Pre-procesamiento de datos](#4)
    - [4.1. - Datos faltantes](#4.1)
    - [4.2. - Data Categóricos](#4.2)
    - [4.2. - Balanceo de clases](#4.3)
    - [4.4. - Pre-Procesamiento especial para NLP](#4.4)
- [5 - Modelos](#5)
- [6 - Ajuste de hiperparámetros](#6)
- [7 - Conclusiones](#7)
- [8 - Referencias](#8)

<a name="1"></a>
## 1. Objetivos

Practicar con un problema de procesamiento del lenguaje natural.
<br>
Aquí, dado un conjunto de tweets, analizar si el sentimiento es positivo o negativo

<a name="2"></a>
## 2. Librerías necesarias

In [3]:
# que no se impriman info y warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [29]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tensorflow.keras import layers,callbacks,models,Sequential,losses
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from keras import backend as K
import os,random
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers.experimental import preprocessing
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import csv
from datetime import datetime
from sklearn.model_selection import train_test_split 
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

<a name="3"></a>
## 3. Carga y visualización de datos

Tenemos 2 datasets, uno para entrenamiento y otro para test:

In [88]:
train_data_pandas = pd.read_csv('data/Corona_NLP_train.csv',encoding='latin-1')
test_data_pandas = pd.read_csv('data/Corona_NLP_test.csv',encoding='latin-1')

In [89]:
train_data_pandas.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


<a name="4"></a>
## 4. Pre-procesamiento de datos

<a name="4.1."></a>
### 4.1. Datos faltantes

In [90]:
print(f'Datos faltantes train:\n {train_data_pandas.isnull().sum()} \n')
print(f'Datos faltantes test:\n {test_data_pandas.isnull().sum()}')

Datos faltantes train:
 UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64 

Datos faltantes test:
 UserName           0
ScreenName         0
Location         834
TweetAt            0
OriginalTweet      0
Sentiment          0
dtype: int64


Vemos que no hay datos faltantes en las columnas que nos interesan (OriginalTweet y Sentiment).

<a name="4.2"></a>
### 4.2. Datos categóricos 

Nos interesaremos en las columnas OriginalTweet y Sentiment; a su vez veremos las distintas opciones de esta última columna:

In [91]:
train_data_pandas['Sentiment'].unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

Convertiremos esta columna a valores numéricos; a su vez no nos interesa ser tan específicos respecto a si un sentimiento es postivo o extremadamente positivo, más bien distinguiremos entre positivo y negativo. Los neutrales los consideraremos positivos. 

In [92]:
label_map = {'Extremely Negative':0,'Negative':0,'Neutral':1,'Positive':1,'Extremely Positive':1}
train_data_pandas['Sentiment'] = train_data_pandas['Sentiment'].map(label_map)
test_data_pandas['Sentiment'] = test_data_pandas['Sentiment'].map(label_map)

Chequeamos

In [93]:
train_data_pandas['Sentiment'].unique()

array([1, 0])

<a name="4.3"></a>
### 4.3.  Balanceo de clases

Veamos si las clases están balanceadas.

In [94]:
train_data_pandas['Sentiment'].value_counts()

1    25759
0    15398
Name: Sentiment, dtype: int64

In [95]:
25759/(25759+15398)

0.625871662171684

Tenemos un desbalance moderado.

<a name="4.4"> </a>
### 4.4. Pre-procesamiento especial para NLP

Vamos a pe-procesar el texto de OriginalTweet, para ello:


- Quitaremos las stop-words
- Quitaremos algunos caracteres especiales, como "@"
- Aplicaremos Lemmatization


<b>Nota:</b> Habría que quitar también las puntuaciones, llevar todo a minúscula y tokenizar, pero eso lo haremos luego con TextVectorization.

Descargaremos e imprimiremos para ver las stop words:

In [96]:
nltk.download('stopwords')
# View stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /home/marcos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


También necesitaremos "punkt" y "wordnet":

In [97]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/marcos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/marcos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Quitamos las stop words y aplicaremos la lematización:

In [98]:
# Inicializar lematizador
lemmatizer = WordNetLemmatizer()

# stop words
stop_words = set(stopwords.words('english'))

# Función para quitar palabras de parada y lematizar un texto
def preprocess_text(text):
    words = word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Aplicar la función a la columna 'OriginalTweet' del dataset, tanto en train como test
train_data_pandas['OriginalTweet'] = train_data_pandas['OriginalTweet'].apply(preprocess_text)
test_data_pandas['OriginalTweet'] = test_data_pandas['OriginalTweet'].apply(preprocess_text)

Además vamos a eliminar caracteres especiales, como @ y # (nos quedaremos con otros, como "!", pues pueden ser importantes para el significado).

In [99]:
# Eliminar arrobas en direcciones de correo electrónico o menciones
def preprocess_text2(text):
    return re.sub(r'[@#]', '', text) #&

# lo aplicamos
train_data_pandas['OriginalTweet'] = train_data_pandas['OriginalTweet'].apply(preprocess_text2)
test_data_pandas['OriginalTweet'] = test_data_pandas['OriginalTweet'].apply(preprocess_text2)

Veamos cómo quedaron los datos:

In [100]:
train_data_pandas['OriginalTweet']

0         menyrbie  phil_gahan  chrisitv http : //t.co/...
1        advice talk neighbour family exchange phone nu...
2        coronavirus australia : woolworth give elderly...
3        food stock one empty ... please , n't panic , ...
4        , ready go supermarket  covid19 outbreak . 'm ...
                               ...                        
41152    airline pilot offering stock supermarket shelf...
41153    response complaint provided citing covid-19 re...
41154    know itâs getting tough  kameronwilds rationi...
41155    wrong smell hand sanitizer starting turn ?  co...
41156     tartiicat well new/used rift going $ 700.00 a...
Name: OriginalTweet, Length: 41157, dtype: object

#### Dividimos en train/val

In [102]:
# Dividir los datos en conjuntos de entrenamiento y validación
train_data, val_data = train_test_split(train_data_pandas, test_size=0.2, random_state=42)


#### Vamos ahora a crear los datasets para trabajar con tensorflow

In [103]:
# Cargar el DataFrame de pandas en un objeto tf.data.Dataset
# armo según lo que me interesa
train_dataset = tf.data.Dataset.from_tensor_slices((train_data['OriginalTweet'].values, 
                                              train_data['Sentiment'].values))



validation_dataset = tf.data.Dataset.from_tensor_slices((val_data['OriginalTweet'].values, 
                                              val_data['Sentiment'].values))



In [104]:
# veo un dato de train y uno de test
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())
    

text:  b'unemployment claim made online virginia week : monday : 426 tuesday : 2,150 number going get bigger . http : //t.co/fueg2rl2dl'
label:  0


In [105]:
# Repetimos para test

# Cargar el DataFrame de pandas en un objeto tf.data.Dataset
test_dataset = tf.data.Dataset.from_tensor_slices((test_data_pandas['OriginalTweet'].values, 
                                                   test_data_pandas['Sentiment'].values))

Definimos el tamaño del buffer y del lote:

In [106]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [107]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Veamos algunos ejemplos y sus etiquetas

In [108]:
for example, label in train_dataset.take(1):
  print(f'texts:  {example.numpy()[:3]}\n')
  print(f'labels: , {label.numpy()[:3]}')

texts:  [b'support initiative uae government contain spread covid-19 ,  lulumallfujairah temporarily closed mar 25th notice . period lulu hypermarket & amp ; pharmacy open . restaurant serve home delivery . http : //t.co/tygpevpams'
 b'lysol laundry sanitizer additive , crisp linen - 90oz http : //t.co/4nxnk307wx  coronavirus  covid_19  covid19 http : //t.co/tkwhznbhwm'
 b'read tip keep  coronavirus  scammer bay . http : //t.co/am6qpqx249 remain vigilant  cyberaware  cybersecurity http : //t.co/qhcpdgsdrt']

labels: , [1 1 1]


Ahora crearemos y aplicaremos una capa llamada <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization" target='_blanck'>TextVectorization</a>, que quitará las puntuaciones, pasará todo a minúsculas y tokenizará:

In [109]:
VOCAB_SIZE = 10000
max_length = 45 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)


encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE,
                                    output_mode="int",
                                    output_sequence_length=max_length)

# Fit the text vectorizer instance to the training data using the adapt() method
encoder.adapt(train_dataset.map(lambda text, label: text))


A continuación se muestran los primeros 20 tokens:

In [110]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'http', 'coronavirus', 'covid19', 'price', 'store',
       'supermarket', 'food', 'grocery', 'people', 'amp', 'consumer',
       '19', 'covid', 'shopping', 's', 'online', 'need', 'time'],
      dtype='<U27')

Ahora que el vocabulario está configurado, la capa puede codificar el texto en índices. Los tensores de índices son rellenados con 0s para que tengan el tamaño de la secuencia más larga en el lote.

Veamos un ejemplo

In [111]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 127, 1868, 2314,   97, 1641,   91,    4,    1,  831,  217, 1895,
        5144,  840,  530, 4997, 2579,   11,  294,  146,  189, 1219,   28,
          57,    2,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0],
       [1544, 2331,   30, 4229, 4391, 5329,    1,    2,    1,    3,    4,
           4,    2,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0],
       [ 131,  270,   61,    3,  521, 1672,    2,    1,  502, 2683,    1,
        3447,    2,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]])

Vemos que rellena con 0s hasta tener siempre un largo de 45.

Con esta configuración, el proceso no es completamente reversible (no hay un mapeo uno a uno)

In [112]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'support initiative uae government contain spread covid-19 ,  lulumallfujairah temporarily closed mar 25th notice . period lulu hypermarket & amp ; pharmacy open . restaurant serve home delivery . http : //t.co/tygpevpams'
Round-trip:  support initiative uae government contain spread covid19 [UNK] temporarily closed mar 25th notice period lulu hypermarket amp pharmacy open restaurant serve home delivery http [UNK]                    

Original:  b'lysol laundry sanitizer additive , crisp linen - 90oz http : //t.co/4nxnk307wx  coronavirus  covid_19  covid19 http : //t.co/tkwhznbhwm'
Round-trip:  lysol laundry sanitizer additive crisp linen [UNK] http [UNK] coronavirus covid19 covid19 http [UNK]                               

Original:  b'read tip keep  coronavirus  scammer bay . http : //t.co/am6qpqx249 remain vigilant  cyberaware  cybersecurity http : //t.co/qhcpdgsdrt'
Round-trip:  read tip keep coronavirus scammer bay http [UNK] remain vigilant [UNK] cybersecurity http [

Puede observarse que hay muchos tokens desconocidos ([UNK])

Finalmente, aplicaremos una capa de embedding.

In [113]:

embedding = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, # set the input shape; size of our vocabulary
                                 output_dim=128, # set the size of the embedding vector
                                 embeddings_initializer="uniform", # default, initialize embedding vectors randomly
                                 input_length=max_length # how long is each input
                             )

embedding

<a name="5"> </a>
## MODELOS

Probaremos distinos modelos.

### Baseline

Es un modelo muy simple con Scikit Learn.

In [114]:


inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = encoder(inputs) # turn the input text into numbers 
x = embedding(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model



In [116]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_dataset,
                        #train_labels, 
                        epochs=5,
                        validation_data=validation_dataset)
                        #callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                        #                                       experiment_name="model_1_dense")])

Epoch 1/5
515/515 [==============================] - 9s 17ms/step - loss: 0.2312 - accuracy: 0.9169 - val_loss: 0.3703 - val_accuracy: 0.8543
Epoch 2/5
515/515 [==============================] - 9s 17ms/step - loss: 0.2149 - accuracy: 0.9230 - val_loss: 0.3793 - val_accuracy: 0.8545
Epoch 3/5
515/515 [==============================] - 9s 17ms/step - loss: 0.2018 - accuracy: 0.9292 - val_loss: 0.3918 - val_accuracy: 0.8542
Epoch 4/5
515/515 [==============================] - 10s 20ms/step - loss: 0.1911 - accuracy: 0.9339 - val_loss: 0.4009 - val_accuracy: 0.8513
Epoch 5/5
515/515 [==============================] - 11s 21ms/step - loss: 0.1817 - accuracy: 0.9367 - val_loss: 0.4127 - val_accuracy: 0.8525


In [120]:
for x,y in validation_dataset:
    print(y)

tf.Tensor(
[0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 0 0 0
 0 1 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1], shape=(64,), dtype=int64)
tf.Tensor(
[0 1 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 0 1 0 1 0 1 1 0
 1 0 0 0 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 0 1 0 1], shape=(64,), dtype=int64)
tf.Tensor(
[1 1 1 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 0 0 0 1 1 0 1 1 0 1
 0 0 1 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1], shape=(64,), dtype=int64)
tf.Tensor(
[1 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0
 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 0], shape=(64,), dtype=int64)
tf.Tensor(
[0 1 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 0 1 0 0 0 1 1 1 0 0 0 1 1
 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 1 0 1 0], shape=(64,), dtype=int64)
tf.Tensor(
[0 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 1 0
 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 1 1 0 0 1 1], shape=(64,),

In [121]:
for text, label in validation_dataset:
    print("Etiqueta:", label.numpy())

Etiqueta: [1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0
 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1]
Etiqueta: [0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 1
 1 0 1 0 0 0 1 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 1 1 1 1 1]
Etiqueta: [1 1 1 0 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 1
 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 1 0 1 1 0 0 1 1 0]
Etiqueta: [1 0 1 1 0 0 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 1 0 1 1 1 1 1 1 1
 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1]
Etiqueta: [0 1 1 1 1 1 1 0 0 1 0 1 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0
 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 0 0 0 0 1 1 0 1]
Etiqueta: [1 1 0 1 1 0 0 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 0 1
 1 1 0 1 1 0 1 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1 1 1 1 1 1]
Etiqueta: [1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 1 1
 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1]
Etiqueta: [0 

In [128]:
model_1.evaluate(test_dataset)

60/60 [==============================] - 0s 2ms/step - loss: 0.4691 - accuracy: 0.8344


[0.46907660365104675, 0.8343865275382996]

In [138]:





sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie')
predictions = model_1.predict(np.array([sample_text]))

predictions


binary_prediction = 1 if predictions[0, 0] > 0.5 else 0


binary_prediction

1/1 [==============================] - 0s 17ms/step


1

In [161]:


from sklearn.metrics import confusion_matrix, classification_report

# Supongamos que ya has entrenado tu modelo y tienes un modelo llamado 'modelo_entrenado'

# Crear listas para almacenar las etiquetas verdaderas y las predicciones del modelo
true_labels = []
predicted_labels = []

# Iterar sobre el conjunto de validación y realizar predicciones
for text, label in validation_dataset:
    # Preprocesar el texto (si es necesario)
    # ...
    
    
    print(type(text))
    


    # Realizar la predicción con el modelo entrenado
    predictions = model_1.predict(text.numpy(),verbose=0)
    
    binary_prediction = 1 if predictions[0, 0] > 0.5 else 0
    
    
    #sample_text = ('The movie was cool. The animation and the graphics '
    #               'were out of this world. I would recommend this movie')
    #predictions = model_1.predict(np.array([sample_text]))

    # Convertir las predicciones a etiquetas (por ejemplo, usando argmax)
    predicted_label = tf.argmax(predictions).numpy()

    # Almacenar las etiquetas verdaderas y las predicciones
    true_labels.append(label.numpy())
    
    predicted_labels.append(binary_prediction)
    
    #break

    

    
predicted_labels    



print(true_labels)

# Calcular la matriz de confusión y otras métricas
#conf_matrix = confusion_matrix(true_labels, predicted_labels)
#classification_rep = classification_report(true_labels, predicted_labels)

# Imprimir la matriz de confusión y el informe de clasificación
#print("Matriz de Confusión:")
#print(conf_matrix)

#print("\nInforme de Clasificación:")
#print(classification_rep)



<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.fr

In [167]:





"""
EN LA INFERENCIA NO NECESITO APLICAR
ENCODE Y EMBEDDING XQ LO HAGO
DIRECTAMENTE EN EL MODELO
"""



import tensorflow as tf

# Supongamos que ya has definido tu modelo (model_1), el encoder y la capa de embedding

# Texto de entrada para hacer una predicción
input_text = "some text"



input_text = preprocess_text(input_text)

input_text = preprocess_text2(input_text)





pred = model_1.predict(np.array([input_text]))

print(pred)



# Convertir la salida a una predicción binaria (0 o 1) si es necesario
binary_prediction = 1 if pred[0, 0] > 0.5 else 0

# Imprimir la predicción
print("Predicción:", binary_prediction)


1/1 [==============================] - 0s 16ms/step
[[0.9017605]]
Predicción: 1
